In [3]:
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!ls /content/drive/MyDrive/

In [ ]:
from google.colab import files
uploaded = files.upload()  #Открываем окно загрузки файлов


In [ ]:
import shutil

shutil.move('train.csv', '/content/drive/MyDrive/Арматура/train.csv')
shutil.move('test.csv', '/content/drive/MyDrive/Арматура/test.csv')


In [ ]:
!ls /content/drive/MyDrive/Арматура/


In [ ]:
import pandas as pd

train_data = pd.read_csv('/content/drive/MyDrive/Арматура/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Арматура/test.csv')

print(train_data.head())  #Вывод первых строк


In [ ]:
train_data['date'] = pd.to_datetime(train_data['date'])  #Преобразуем в дату
train_data.set_index('date', inplace=True)  #Делаем индексом дату
train_weekly = train_data.resample('W-MON').mean()  #Среднее значение по понедельникам

print(train_weekly.head())  #Проверяем результат


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.plot(train_weekly.index, train_weekly['price'], label='Цена арматуры')
plt.xlabel('Дата')
plt.ylabel('Цена')
plt.legend()
plt.grid()
plt.show()


In [ ]:
train_size = int(len(train_weekly) * 0.8)  #80% данных для обучения
train, test = train_weekly[:train_size], train_weekly[train_size:]

print(f"Обучающая выборка: {len(train)} недель, Тестовая: {len(test)} недель")


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

#Обучаем модель
model = ARIMA(train['price'], order=(3,1,2))  #Параметры можно оптимизировать
model_fit = model.fit()

#Прогнозируем на тестовой выборке
forecast = model_fit.forecast(steps=len(test))

#Визуализируем прогноз
plt.figure(figsize=(12,6))
plt.plot(train.index, train['price'], label='Обучение')
plt.plot(test.index, test['price'], label='Истинные данные', color='orange')
plt.plot(test.index, forecast, label='Прогноз', linestyle='dashed', color='red')
plt.xlabel('Дата')
plt.ylabel('Цена')
plt.legend()
plt.grid()
plt.show()


In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

#Создаем дополнительные признаки, скользящее среднее
weekly_prices['Moving_Avg'] = weekly_prices['Price'].rolling(window=4).mean()

#Создаем набор данных для обучения
X = weekly_prices[['Price', 'Moving_Avg']]
y = weekly_prices['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = CatBoostRegressor(iterations=500, depth=10, learning_rate=0.1)
model.fit(X_train, y_train)

#Прогнозирование
y_pred = model.predict(X_test)


In [ ]:
#Прогноз на ближайшие недели (пример)
predicted_prices = forecast  #Сюда подставьте результат прогноза

#Принятие решения на основе прогноза
if predicted_prices[0] > weekly_prices['Price'].iloc[-1]:  #Цена через неделю выше текущей
    tender_weeks = 3  #Закупаем на 3 недели
elif predicted_prices[0] < weekly_prices['Price'].iloc[-1]:  #Цена через неделю ниже
    tender_weeks = 1  #Закупаем на 1 неделю
else:
    tender_weeks = 2  #Закупаем на 2 недели, если цена стабильна

print(f"Рекомендованный объем тендера: {tender_weeks} недель.")


In [ ]:
from sklearn.metrics import mean_absolute_error

#Оценка модели
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')
